## Scraping transcripts from Google Docs

[The Adventure Zone](https://www.maximumfun.org/shows/adventure-zone) is a Dungeons and Dragons podcast that is enjoyable even to those who have never played DnD (like me). This notebook lays out how I downloaded the transcripts for some language analysis. Huge thanks to the people behind the volunteer [transcribing project](http://tazscripts.tumblr.com/). It's a great resource for anyone in the Deaf community or people like me who want to download all of the words that have been spoken to process.

#### Written in Python

In [1]:
import os
import requests
import re
import pandas as pd

from bs4 import BeautifulSoup

The transripts are all hosted on Google docs for community editing. The [master tracking document](https://docs.google.com/document/d/1zVGa1zgr7BjMC_0L06_XpmuIn2jgDKHvPZkohHy0YrA/edit) has links to every transcript and what percent is completed. Scraping a google doc is a little harder than a web page. I copied the tracking document to my own google drive and published it to the web so that I could scrape all of the transcript links from it.

In [5]:
#Get html, find all table rows 'tr'
tracking_doc_url = "https://docs.google.com/document/d/e/2PACX-1vT5IOvuFz988qAbP2P5ELDIekf4mUdfBsF1LGGW8CxMcry0YjJ34H1iFYN1qx0B7eYPUzw4CiSd8kbM/pub"
tracking_html = requests.get(tracking_doc_url)
tracking_beautiful = BeautifulSoup(tracking_html.text, "lxml")
rows = tracking_beautiful.findAll('tr')

In [6]:
completed_links = {}
for row in rows:
    #strip row.text because there is some trailing whitespace
    if row.text.strip().endswith('100%'):
        a = row.find('a')
        completed_links[a.text] = a['href']

I need to pull the Google doc ID out of every link and request an auto download version of the Google doc (see [this site](http://jessicastansberry.com/googledrive/) for info on that)

In [7]:
def google_doc_download(docID, download_as='txt'):
    return 'https://docs.google.com/document/d/' + docID + '/export?format=' + download_as

In [8]:
#This loop will take a while to download all of the episodes
os.mkdir('data')
for episode, link in completed_links.items():
    
    try:
        #regex search to pull the ID out of the link
        docID = re.search(string=link, pattern=r'(?:/d/|id%3D)(.*?)(?:/|&)')[1]
    except:
        print(f"{episode} didn't download ({link})")
        continue
        
    download_link = google_doc_download(docID, download_as='txt')
    
    with open(os.path.join(os.getcwd(), 'data', episode + '.txt'), 'w') as openfile:
        document = requests.get(download_link)
        openfile.write(document.text)


### Now that all of the files are downloaded we can parse them into csv files
Loop through all of the downloaded text files and read them into a pandas DataFrame. The speaker and line can be split using a colon as a delimiter. More than one colon is present in some lines and not in others, so this has to be done after the file is loaded in. After expanding the DataFrame on colons, the lines can be put back together. I saved every file to a separate csv to hang onto them separately.

In [9]:
docs = os.listdir(os.path.join(os.getcwd(), 'data'))

In [10]:
os.mkdir('export')
for script in docs:
    with open(os.path.join(os.getcwd(), 'data', script)) as myfile:
        onefile = pd.read_csv(myfile, sep='\n', header=None)
    
    #split on colon
    onefile = pd.DataFrame(onefile[0].str.split(':', expand=True))
    onefile = onefile.fillna('')
    #piece line columns back together
    onefile.iloc[:,1] = onefile.iloc[:,1].astype(str).str.cat(onefile.iloc[:,2:])
    onefile = onefile.drop(columns=[i for i in range(2, len(onefile.columns))])
    #see if the episode number can be parsed from the episode title
    try:
        epnum = int(re.search(string=script, pattern=r'[\d]+')[0])
    except:
        #if not use the script name (without '.txt')
        epnum = script[:-4]
    
    onefile['episode'] = epnum
    #set column names
    onefile.columns = ['speaker', 'lines', 'episode']
    #reorder
    onefile = onefile[['episode', 'speaker', 'lines']]
    onefile.to_csv(os.path.join(os.getcwd(), 'export', f'{epnum}.csv'))

### Read all the files in, add to a DataFrame, and save the combined csv file as "TAZ.csv"

In [11]:
csvs = os.listdir(os.path.join(os.getcwd(), 'export'))

df = pd.read_csv(os.path.join(os.getcwd(), 'export', csvs[0]))
df.columns = ['line_num', 'episode', 'speaker', 'lines']
for csv in csvs[1:]:
    newdf = pd.read_csv(os.path.join(os.getcwd(), 'export', csv))
    newdf.columns = ['line_num', 'episode', 'speaker', 'lines']
    df = df.append(newdf)

df = df.sort_values(by=['episode', 'line_num'], ascending=True)
df.reset_index(inplace=True, drop=True)
df.line_num = df.line_num.astype(int)
df.to_csv(os.path.join(os.getcwd(), 'TAZ.csv'))

These scripts are not very clean. The headers of each transcript are still there. There are things in the speaker column like \[laughing] or \[Theme music]. But it is a good stoping point.